In [2]:
import sys
sys.path.insert(0, '..')

In [3]:
from data_formatters.base import GenericDataFormatter, InputTypes, DataTypes
from data_formatters.sorgenia_wind import SorgeniaFormatter
from expt_settings.configs import ExperimentConfig
from libs.hyperparam_opt import HyperparamOptManager
from libs.tft_model import TemporalFusionTransformer
import libs.utils as utils
import os
import pandas as pd
from pandas import DataFrame, Series, Timestamp, Index
from tqdm import tqdm

In [4]:
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf1
from tensorflow.compat.v1 import Session, ConfigProto
from tensorflow.python.eager.context import PhysicalDevice
from typing import Dict, List, Union, Generator
from numpy import load
import plotly.graph_objects as go
from plotly.graph_objects import Figure

IMPORT FUNCTIONS TO TRANSFORM DF AND CALCULATE MAPE

In [5]:
from inference import mape
from inference import utils as infutils
from script_download_data import preprocess_sorgenia
from inference import evaluate

In [6]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [7]:
gpu: List[PhysicalDevice] = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [8]:
# Tensorflow setup
default_keras_session: Session = tf1.keras.backend.get_session()
tf_config: ConfigProto = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)

Selecting GPU ID=0


In [9]:
file_path: str = r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs\data\sorgenia_wind\data\sorgenia_wind\data\sorgenia_final_cop.csv'

# READ RAW DATA

In [10]:
config = ExperimentConfig('sorgenia_wind', r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs')

In [11]:
# extract data into DataFrame
def extract_data(db: bool = False) -> DataFrame:
    if db:
        raw_data: DataFrame = preprocess_sorgenia('copernicus', config, True)
    else:
        raw_data: DataFrame = pd.read_csv(file_path)
    
    return raw_data

In [12]:
raw_data = extract_data(False)
raw_data.head()

,plant_name_up,time,kwh,dew_point_2m_C,temperature_2m_C,msl_pressure_hPa,sfc_pressure_hPa,precipitation_1h_mm,wind_speed_mean_10m_1h_ms,wind_speed_mean_100m_1h_ms,...,days_from_start,id,hour,day,day_of_week,month,categorical_id,hours_from_start,categorical_day_of_week,categorical_hour
0,UP_MPNTLCDMRN_1,2016-12-31 23:00:00+00:00,225.000,-0.902831,4.980909,1028.935552,983.718765,0.000000e+00,1.381208,2.254668,...,0,UP_MPNTLCDMRN_1,23,31,5,12,UP_MPNTLCDMRN_1,0.0,5,23
1,UP_MPNTLCDMRN_1,2017-01-01 00:00:00+00:00,65.625,-0.783022,4.890773,1028.693079,983.524042,-1.734723e-15,1.278206,2.224647,...,0,UP_MPNTLCDMRN_1,0,1,6,1,UP_MPNTLCDMRN_1,1.0,6,0
2,UP_MPNTLCDMRN_1,2017-01-01 01:00:00+00:00,31.875,-0.565645,4.807044,1028.156679,983.034144,-1.734723e-15,1.211802,2.130597,...,0,UP_MPNTLCDMRN_1,1,1,6,1,UP_MPNTLCDMRN_1,2.0,6,1
3,UP_MPNTLCDMRN_1,2017-01-01 02:00:00+00:00,0.000,-0.331075,4.911959,1028.162779,983.062126,-1.734723e-15,1.017035,1.846413,...,0,UP_MPNTLCDMRN_1,2,1,6,1,UP_MPNTLCDMRN_1,3.0,6,2
4,UP_MPNTLCDMRN_1,2017-01-01 03:00:00+00:00,0.000,-0.668040,4.464566,1027.616920,982.551146,-1.734723e-15,0.715094,1.450024,...,0,UP_MPNTLCDMRN_1,3,1,6,1,UP_MPNTLCDMRN_1,4.0,6,3


# OVERRIDE ATTRIBUTE TO SET MODEL FEATURES

In [13]:
def choose_formatter(model: str, config: ExperimentConfig) -> SorgeniaFormatter:
    if model == "no_forecasts":
        formatter: SorgeniaFormatter = config.make_data_formatter()
        formatter._column_definition = [
            ('id', DataTypes.REAL_VALUED, InputTypes.ID),
            ('time', DataTypes.DATE, InputTypes.TIME),
            ('kwh', DataTypes.REAL_VALUED, InputTypes.TARGET),
            ('hour', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),
            ('day_of_week', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),
            ('hours_from_start', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),
            ('categorical_id', DataTypes.CATEGORICAL, InputTypes.STATIC_INPUT),
            ('dew_point_2m_C', DataTypes.REAL_VALUED, InputTypes.OBSERVED_INPUT),
            ('temperature_2m_C', DataTypes.REAL_VALUED, InputTypes.OBSERVED_INPUT),
            ('msl_pressure_hPa', DataTypes.REAL_VALUED, InputTypes.OBSERVED_INPUT),
            ('sfc_pressure_hPa', DataTypes.REAL_VALUED, InputTypes.OBSERVED_INPUT),
            ('precipitation_1h_mm', DataTypes.REAL_VALUED, InputTypes.OBSERVED_INPUT),
            ('wind_speed_mean_10m_1h_ms', DataTypes.REAL_VALUED, InputTypes.OBSERVED_INPUT),
            ('wind_speed_mean_100m_1h_ms', DataTypes.REAL_VALUED, InputTypes.OBSERVED_INPUT),
            ('wind_dir_mean_100m_1h_d', DataTypes.REAL_VALUED, InputTypes.OBSERVED_INPUT),
            ('wind_dir_mean_10m_1h_d', DataTypes.REAL_VALUED, InputTypes.OBSERVED_INPUT),
            ('wind_gusts_10m_1h_ms', DataTypes.REAL_VALUED, InputTypes.OBSERVED_INPUT),
            ('wind_gusts_10m_ms', DataTypes.REAL_VALUED, InputTypes.OBSERVED_INPUT),
            ('Day sin', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),
            ('Day cos', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),
            ('Year sin', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),
            ('Year cos', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),
        ]
        
        return formatter
    
    else:
        # return formatter as configured for model with forecasts
        formatter: SorgeniaFormatter = config.make_data_formatter()
        return formatter     

In [14]:
formatter: SorgeniaFormatter = choose_formatter('forecasts', config)

In [15]:
data_csv_path: str = config.data_csv_path

# SPLIT DATA

In [16]:
train, valid, test = formatter.split_data(raw_data)

Setting scalers with training data...


In [17]:
# Sets up default params
fixed_params: Dict = formatter.get_experiment_params()
params: Dict = formatter.get_default_model_params()
params["model_folder"]: str = os.path.join(config.model_folder, "fixed")
model_folder = os.path.join(config.model_folder, "fixed")

In [18]:
# Sets up hyperparam manager
print("*** Loading hyperparm manager ***")
opt_manager = HyperparamOptManager({k: [params[k]] for k in params},
                                   fixed_params, model_folder)

*** Loading hyperparm manager ***


In [19]:
# predictions_path: str = r'C:\Users\Lorenzo\savedmodels_sorgenia_wind_known'

# GENERATE PREDICTIONS ON TESTSET

In [20]:
evaluate.compute_predictions(test, opt_manager, formatter, config, tf_config, default_keras_session, 'sorgenia_wind')

*** Running tests ***

category counts are  [7]
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.1
# hidden_layer_size = 160
# learning_rate = 0.001
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = C:\Users\Lorenzo\PycharmProjects\TFT\outputs\fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 180
# num_encoder_steps = 168
# num_epochs = 100
# early_stopping_patience = 10
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('time', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('kwh', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('dew_point_2m_C', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('temperature_2m_C', <DataTypes.REAL_VALU

OSError: Unable to open file (unable to open file: name = 'C:\Users\Lorenzo\PycharmProjects\TFT\outputs\fixed\sorgenia_wind_ckpt.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

# CALCULATE MAPE

In [ ]:
df_mape = evaluate.evaluate(opt_manager.hyperparam_folder, 'copernicus_forecasts_model', 700)

# PLOT MAPE BOXPLOT

In [ ]:
fig: Figure = evaluate.boxplotter(df_mape)
fig.show()

# MODEL WITHOUT FORECASTS

In [ ]:
formatter: SorgeniaFormatter = choose_formatter('no_forecasts', config)
train, valid, test = formatter.split_data(raw_data)
# Sets up default params
fixed_params: Dict = formatter.get_experiment_params()
params: Dict = formatter.get_default_model_params()
params["model_folder"]: str = os.path.join(config.model_folder, "fixed")
model_folder = os.path.join(config.model_folder, "fixed")
# Sets up hyperparam manager
print("*** Loading hyperparm manager ***")
opt_manager = HyperparamOptManager({k: [params[k]] for k in params},
                                   fixed_params, model_folder)
evaluate.compute_predictions(test, opt_manager, formatter, config, tf_config, default_keras_session, 'sorgenia_wind_no_forecasts')
df_mape = evaluate.evaluate(opt_manager.hyperparam_folder, 'copernicus_no_forecasts_model', 700)
fig: Figure = evaluate.boxplotter(df_mape)
fig.show()